In [1]:
import glob
import pandas as pd
import numpy as np
import environment2
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from collections import defaultdict
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind
from scipy.stats import wilcoxon
from statsmodels.tsa.stattools import adfuller
import pymannkendall as mk
import statistics

In [2]:
env = environment2.environment2()
user_list_2D = env.user_list_2D

In [3]:
user_list_2D[0]

'data/NDSI-2D\\U_1.csv'

In [4]:
fileNames2D=['data/NDSI-2D\\taskname_ndsi-2d-task_userid_3abeecbe-327a-441e-be2a-0dd3763c1d45.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_44968286-f204-4ad6-a9b5-d95b38e97866.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_6d49fab8-273b-4a91-948b-ecd14556b049.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_72a8d170-77ae-400e-b2a5-de9e1d33a714.csv',
 'data/NDSI-2D\\taskname_ndsi-2d-task_userid_733a1ac5-0b01-485e-9b29-ac33932aa240.csv']

In [5]:
fileNames2D=user_list_2D

In [6]:
fileNames2D

array(['data/NDSI-2D\\U_1.csv', 'data/NDSI-2D\\U_10.csv',
       'data/NDSI-2D\\U_11.csv', 'data/NDSI-2D\\U_12.csv',
       'data/NDSI-2D\\U_13.csv', 'data/NDSI-2D\\U_14.csv',
       'data/NDSI-2D\\U_15.csv', 'data/NDSI-2D\\U_16.csv',
       'data/NDSI-2D\\U_17.csv', 'data/NDSI-2D\\U_18.csv',
       'data/NDSI-2D\\U_19.csv', 'data/NDSI-2D\\U_2.csv',
       'data/NDSI-2D\\U_20.csv', 'data/NDSI-2D\\U_3.csv',
       'data/NDSI-2D\\U_4.csv', 'data/NDSI-2D\\U_5.csv',
       'data/NDSI-2D\\U_6.csv', 'data/NDSI-2D\\U_7.csv',
       'data/NDSI-2D\\U_8.csv', 'data/NDSI-2D\\U_9.csv'], dtype='<U21')

In [7]:
def get_ROISubset(data):
    roi_subset=[]
    subset=0
    for i in range(len(data)):
        if data["State"][i]=='Answering':
            data["State"][i]='Sensemaking'
        # if data['State'][i]=='Sensemaking':
        #     if (i < (len(data) - 1)) and data['State'][i+1]!= 'Sensemaking':
        #         roi_subset.append(subset)
        #         subset=subset+1
        #     else:
        #         roi_subset.append(subset)
        # else:
        #     roi_subset.append(subset)
        if i< int(len(data)/2):
            roi_subset.append(0)
        else:
            roi_subset.append(1)
    return roi_subset



In [8]:
from collections import defaultdict
def get_probabilities(dataframe):
    dataframe['high-level-action']=np.zeros(len(dataframe))
    probabilities=[]
    map=dict()
    # print(dataframe.head(5))
    states=['Sensemaking','Foraging','Navigation']
    actions=['change','same']
    for state in states:
      for action in actions:
        map[state+action]=0


    current_state=dataframe['State'][0]
    for i in range(1,len(dataframe)):
      if dataframe["State"][i]=='Answering':
        dataframe["State"][i]='Sensemaking'
      if dataframe["State"][i]== current_state:
        dataframe["high-level-action"][i-1]="same"
      else:
        dataframe["high-level-action"][i-1]="change"
      current_state=dataframe["State"][i]
    dataframe["high-level-action"][len(dataframe)-1]="same"
    for i in range(len(dataframe)):
      if dataframe['State'][i]=="Navigation":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Navigationchange']  + map['Navigationsame'])

      elif dataframe['State'][i]=="Foraging":
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Foragingchange'] + map['Foragingsame'])

      else:
        map[str(dataframe['State'][i]+dataframe['high-level-action'][i])] += 1
        probs = (map[str(dataframe['State'][i]+dataframe['high-level-action'][i])])/(map['Sensemakingchange'] + map['Sensemakingsame'])
      probabilities.append(probs)
    dataframe['probabilities']=probabilities
    return dataframe, map

In [9]:
class StationarityTests:
    def __init__(self, significance=.05):
        self.SignificanceLevel = significance
        self.pValue = None
        self.isStationary = None
        self.trend= None
    def Mann_Whitney_U_Test(self,timeseries_X,timeseries_Y):
        #check normality
        # result1 =shapiro(timeseries_X)
        # result2 =shapiro(timeseries_Y)
        # px=result1.pvalue
        # py=result2.pvalue

        # if px < self.SignificanceLevel and py < self.SignificanceLevel:
        #     print("X Y not normal",px,py)
        # else:
        #     print("Normal",px,py)
        #return ttest_ind(timeseries_X, timeseries_Y)

        return mannwhitneyu(timeseries_X, timeseries_Y,alternative='two-sided')

    def Wilcoxon_Test(self,timeseries_X, timeseries_Y):

        return wilcoxon(timeseries_X, timeseries_Y)

    def ADF_Stationarity_Test(self, timeseries, printResults = True):
      #Dickey-Fuller test:
      adfTest = adfuller(timeseries, autolag='AIC')

      self.pValue = adfTest[1]

      if (self.pValue<self.SignificanceLevel):
          self.isStationary = True
      else:
          self.isStationary = False

      if printResults:
          dfResults = pd.Series(adfTest[0:4], index=['ADF Test Statistic','P-Value','# Lags Used','# Observations Used'])
          #Add Critical Values
          for key,value in adfTest[4].items():
              dfResults['Critical Value (%s)'%key] = value
          print('Augmented Dickey-Fuller Test Results:')
          print(dfResults)

    def Mann_Kendall_Stationarity_Test(self, timeseries, printResults = True):
      #Mann Kendall Trend Test:
      mktTest = mk.original_test(timeseries,self.SignificanceLevel)

      self.pValue = mktTest[2]
      self.trend= mktTest[0]
      self.isStationary = mktTest[1]

      if printResults:
          dfResults = pd.Series(mktTest[0:3], index=['Trend Type','Stationarity','P-Value'])
          #Add Critical Values
          for key,value in mktTest[3].items():
              dfResults['Critical Value (%s)'%key] = value
          print('Mann Kendall Trend Test:')
          print(dfResults)

In [10]:
user=[]
roi_cycle=[]
all_states=[]
result=[]


output_dataframe=pd.DataFrame(columns=['User','ROI_Cycle','State','Result'])
for u in fileNames2D:
  print("###########################",u)
  index=[]

  all_dfs=[]
  maps=[]
  df = pd.read_csv(u)

  u=u.lstrip('data/NDSI-2D\\')
  u=u.rstrip('.csv')



  #df['ROI_Cycle']=get_ROISubset(df)
  df['ROI_Cycle']=get_ROISubset(df)
  states=["Sensemaking",'Foraging','Navigation']
  serieses=np.empty(shape=(len(df.ROI_Cycle.unique()),len(states)),dtype=object)
  for k in df.ROI_Cycle.unique():
    dfs,mappings=get_probabilities((df.loc[df['ROI_Cycle'] == k]).reset_index(drop=True))
    for j in range(len(states)):

        state_df=dfs.loc[(dfs['State'] == states[j])]
        state_df.reset_index(inplace=True)

        same_probs=[]
        for indx in range(len(state_df)):
              if state_df['high-level-action'][indx]=='same':
                  same_probs.append(state_df['probabilities'][indx])
              else:
                  same_probs.append(1-state_df['probabilities'][indx])

        serieses[k][j]=same_probs

  for id in range(serieses.shape[1]):
      combinations=[]
      for first_cycle in range(serieses.shape[0]):
          for cycle in range(serieses.shape[0]):
            if first_cycle != cycle and list([first_cycle, cycle]) not in combinations and list(reversed([first_cycle, cycle])) not in combinations:
              print("Roi Cycles: #######################",[first_cycle,cycle]," ########State######## ",states[id])
              #for final dataframe
              all_states.append(states[id])
              user.append(u)
              roi_cycle.append([first_cycle,cycle])
              combinations.append([first_cycle,cycle])
              x=list(serieses[first_cycle][id])
              y=list(serieses[cycle][id])
              # print("Initial Length: ",len(x),len(y))
              #
              # if len(x) == 0:
              #   x = [0.5] * len(y)
              # elif len(y) == 0:
              #   y = [0.5]* len(x)
              # else:
              #     print(" ")

              # if len(y) < len(x):
              #  y = y + statistics.median(x) * (len(x) - len(y))
              # elif len(x) < len(y):
              #  x = x + statistics.median(y) * (len(y) - len(x))
              # else:
              #     print("\n")

              if  (len(x)>2 and len(y)>2):
                        sTest_state = StationarityTests()
                        trend=sTest_state.Mann_Whitney_U_Test(x, y)
                        print("State is: ", states[id], "  MWUT Result Non-Stationarity:  ", trend.pvalue < 0.05, trend.pvalue)
                        result.append(trend.pvalue < 0.05)
              else :
                         print("State is: ", states[id], "  MWUT Result is:  ","Not Enough Data")
                         result.append('NED')
            else:
                print("already calculated")

output_dataframe['User']=user
output_dataframe['Result']=result
output_dataframe['State']=all_states
output_dataframe['ROI_Cycle']=roi_cycle


########################### data/NDSI-2D\U_1.csv
already calculated
Roi Cycles: ####################### [0, 1]  ########State########  Sensemaking
State is:  Sensemaking   MWUT Result Non-Stationarity:   True 0.02770872045387099
already calculated
already calculated
already calculated
Roi Cycles: ####################### [0, 1]  ########State########  Foraging
State is:  Foraging   MWUT Result Non-Stationarity:   False 0.3257087414834152
already calculated
already calculated
already calculated
Roi Cycles: ####################### [0, 1]  ########State########  Navigation
State is:  Navigation   MWUT Result Non-Stationarity:   False 0.22068370676092086
already calculated
already calculated
########################### data/NDSI-2D\U_10.csv
already calculated
Roi Cycles: ####################### [0, 1]  ########State########  Sensemaking
State is:  Sensemaking   MWUT Result Non-Stationarity:   True 6.388097816408855e-05
already calculated
already calculated
already calculated
Roi Cycles: ###

In [11]:
output_dataframe.head(20)

,User,ROI_Cycle,State,Result
0,U_1,"[0, 1]",Sensemaking,True
1,U_1,"[0, 1]",Foraging,False
2,U_1,"[0, 1]",Navigation,False
3,U_10,"[0, 1]",Sensemaking,True
4,U_10,"[0, 1]",Foraging,True
5,U_10,"[0, 1]",Navigation,True
6,U_11,"[0, 1]",Sensemaking,NED
7,U_11,"[0, 1]",Foraging,NED
8,U_11,"[0, 1]",Navigation,True
9,U_12,"[0, 1]",Sensemaking,False


In [12]:
output_dataframe.to_csv("data/NDSI-2D\\roi-manwhitneyresults.csv", index=False)
#output_dataframe.to_csv("data/NDSI-3D\\probability_distribution_roi_cycle_3d.csv", index=False)